# Part 3

In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

In [2]:
username = "user"
password = "grad5100user"
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
port = 3306

In [3]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [4]:
def run_sql(sql, engine):
    with engine.connect() as conn:
        return pd.read_sql(sql, conn)

## Task 1: Filter `chefdetails` and `judges` Tables

In [5]:
# Load entire tables into DataFrames
chefdetails_df = pd.read_sql("SELECT * FROM chefdetails", engine)
judges_df = pd.read_sql("SELECT * FROM judges", engine)

# Filter DataFrames
chefdetails_filtered = chefdetails_df[~chefdetails_df['season'].str.contains('Masters') & (chefdetails_df['season'] != 'Canada 6')]
judges_filtered = judges_df[~judges_df['season'].str.contains('Masters') & (judges_df['season'] != 'Canada 6')]

print(chefdetails_filtered.head())
print(judges_filtered.head())

   num            name        chef hometown  city state   age  \
0    0   Richard Blais  Richard B.     None  None  None  38.0   
1    1   Mike Isabella     Mike I.     None  None  None  35.0   
2    2  Antonia Lofaso  Antonia L.     None  None  None  34.0   
3    3   Tiffany Derry  Tiffany D.     None  None  None  27.0   
4    4      Carla Hall    Carla H.     None  None  None  46.0   

                season  seasonNumber series  placement personOfColor  \
0  All Stars: New York             8     US          1          None   
1  All Stars: New York             8     US          2          None   
2  All Stars: New York             8     US          3          None   
3  All Stars: New York             8     US          4           POC   
4  All Stars: New York             8     US          5           POC   

             occupation  gender  
0                 Owner    Male  
1  Executive Chef/Owner    Male  
2        Executive Chef  Female  
3        Executive Chef  Female  
4     

## Task 2: Trim Tables to Include Only Relevant Columns

In [6]:
# Select relevant columns
chefdetails_trimmed = chefdetails_filtered[['season', 'seasonNumber', 'name', 'placement', 'gender']]
judges_trimmed = judges_filtered[['season', 'seasonNumber', 'episode', 'challengeType', 'guestJudge', 'competedOnTC']]

print(chefdetails_trimmed.head())
print(judges_trimmed.head())

                season  seasonNumber            name  placement  gender
0  All Stars: New York             8   Richard Blais          1    Male
1  All Stars: New York             8   Mike Isabella          2    Male
2  All Stars: New York             8  Antonia Lofaso          3  Female
3  All Stars: New York             8   Tiffany Derry          4  Female
4  All Stars: New York             8      Carla Hall          5  Female
                season  seasonNumber  episode challengeType        guestJudge  \
0  All Stars: New York             8        1     Quickfire     Tom Colicchio   
1  All Stars: New York             8        1   Elimination  Anthony Bourdain   
2  All Stars: New York             8        2     Quickfire         Joe Jonas   
3  All Stars: New York             8        2   Elimination         Katie Lee   
4  All Stars: New York             8        3     Quickfire       David Chang   

  competedOnTC  
0         None  
1         None  
2         None  
3         Non

## Task 3: Create a Table with `season` and `seasonNumber`

In [7]:
season_info_df = chefdetails_trimmed[['season', 'seasonNumber']].drop_duplicates().sort_values(by='seasonNumber')
print(season_info_df)

                  season  seasonNumber
295        San Francisco             1
190          Los Angeles             2
229                Miami             3
93               Chicago             4
263             New York             5
173            Las Vegas             6
109                 D.C.             7
0    All Stars: New York             8
325                Texas             9
307              Seattle            10
244          New Orleans            11
33                Boston            12
49            California            13
77            Charleston            14
126             Colorado            15
157             Kentucky            16
18        All-Stars L.A.            17
280             Portland            18
142              Houston            19
375      World All Stars            20


## Task 4: Answering the Questions

### 4a. Count of Male and Female Chefs

In [8]:
gender_count = chefdetails_filtered['gender'].value_counts().reset_index()
gender_count.columns = ['gender', 'count']
print(gender_count)

   gender  count
0    Male    170
1  Female    152


### 4b. Count of Male and Female Winners

In [9]:
winners_gender = chefdetails_filtered[chefdetails_filtered['placement'] == 1]['gender'].value_counts().reset_index()
winners_gender.columns = ['gender', 'count']
print(winners_gender)

   gender  count
0    Male     14
1  Female      6


### 4c. Count in Top 3 Finalists

In [10]:
top3_gender = chefdetails_filtered[chefdetails_filtered['placement'].isin([1, 2, 3])]['gender'].value_counts().reset_index()
top3_gender.columns = ['gender', 'count']
print(top3_gender)

   gender  count
0    Male     36
1  Female     25


In [11]:
# Calculating the proportions
female_winners = 6
female_top3 = 25

male_winners = 14
male_top3 = 36

# Proportion of female winners among top 3 finalists
proportion_female_winners = female_winners / female_top3

# Overall proportion of winners among top 3 finalists
total_winners = male_winners + female_winners
total_top3 = male_top3 + female_top3
overall_proportion_winners = total_winners / total_top3

proportion_female_winners, overall_proportion_winners

(0.24, 0.32786885245901637)

The proportion of female winners among the top 3 finalists is approximately 24%, while the overall proportion of winners (both male and female) among the top 3 finalists is about 32.8%.

This suggests that female contestants who make it to the top 3 are less likely to win compared to the overall likelihood of a top 3 contestant winning, regardless of gender. While 32.8% of the top 3 contestants win (as expected, since one out of three wins), only 24% of female contestants in the top 3 end up winning.

However, interpreting this difference as significant would require a more detailed statistical analysis. Factors to consider include the sample size, the competitive dynamics of each season, and other potential biases. Without a deeper statistical test (like a chi-square test for independence), it's hard to definitively say whether this is due to chance or indicates a systemic pattern. But on the surface, the lower proportion of female winners among top finalists could be seen as indicative of a trend worth exploring further.

## Task 5: Table for First Place Winners and Guest Judge Status

In [12]:
# Filter for first place winners
winners = chefdetails_filtered[chefdetails_filtered['placement'] == 1]

# Merge with judges_filtered to check for guest judge status
winners_guest_judge = pd.merge(winners, judges_filtered, how='left', left_on=['name', 'seasonNumber'], right_on=['guestJudge', 'seasonNumber'])

# Determine if each winner was a guest judge
winners_guest_judge['GuestJudge'] = winners_guest_judge['guestJudge'].notna().replace({True: 'Yes', False: 'No'})

# Select and rename relevant columns for the final output
winners_guest_judge_final = winners_guest_judge[['name', 'season_x', 'GuestJudge']].rename(columns={'season_x': 'season'})

print(winners_guest_judge_final)

                    name               season GuestJudge
0          Richard Blais  All Stars: New York         No
1           Melissa King       All-Stars L.A.         No
2                Mei Lin               Boston         No
3            Jeremy Ford           California         No
4      Brooke Williamson           Charleston         No
5        Stephanie Izard              Chicago         No
6           Kevin Sbraga                 D.C.         No
7           Joseph Flamm             Colorado         No
8             Kah-wai Lo              Houston         No
9   Kelsey Barnard Clark             Kentucky         No
10     Michael Voltaggio            Las Vegas         No
11             Ilan Hall          Los Angeles         No
12            Hung Huynh                Miami         No
13         Nicholas Elmi          New Orleans         No
14       Hosea Rosenberg             New York         No
15           Gabe Erales             Portland         No
16       Harold Dieterle       

## Task 6: Non-Winning Contestants Who Were Guest Judges

In [13]:
# Filter for contestants who did not win first place
non_winners = chefdetails_filtered[chefdetails_filtered['placement'] != 1]

# Merge non-winners with the judges dataset to find those who served as guest judges
non_winners_guest_judges = pd.merge(non_winners, judges_filtered, how='inner', left_on='name', right_on='guestJudge')

# From the merged dataset, select only the 'name' column and remove any duplicates
non_winners_guest_judges_unique = non_winners_guest_judges[['name']].drop_duplicates()

print(non_winners_guest_judges_unique)

                  name
0        Mike Isabella
2       Antonia Lofaso
4        Tiffany Derry
8           Dale Talde
10    Spike Mendelsohn
12    Stephen Asprinio
14     Bryan Voltaggio
22     Gregory Gourdet
24     Karen Akunowicz
26       Lee Anne Wong
29         Nini Nguyen
31        Melissa King
34        Amar Santana
36      Kwame Onwuachi
40      Sheldon Simeon
42          Sam Talbot
46       Richard Blais
53   Adrienne Cheatham
54        Carrie Baird
55  Josie Smith-Malave
57        Nina Compton
58      Stefan Richter
60        Dawn Burrell
64      Shota Nakajima
65      Miguel Morales
66   Brooke Williamson
74          Edward Lee
76    Nyesha Arrington
